In [16]:
!unzip data.zip
%ls

Archive:  data.zip
   creating: data/
   creating: data/normal/
  inflating: data/normal/1.jpg       
  inflating: data/normal/10.jpg      
  inflating: data/normal/100.jpg     
  inflating: data/normal/11.jpg      
  inflating: data/normal/12.jpg      
  inflating: data/normal/13.jpg      
  inflating: data/normal/14.jpg      
  inflating: data/normal/15.jpg      
  inflating: data/normal/16.jpg      
  inflating: data/normal/17.jpg      
  inflating: data/normal/18.jpg      
  inflating: data/normal/19.jpg      
  inflating: data/normal/2.jpg       
  inflating: data/normal/20.jpg      
  inflating: data/normal/21.jpg      
  inflating: data/normal/22.jpg      
  inflating: data/normal/23.jpg      
  inflating: data/normal/24.jpg      
  inflating: data/normal/25.jpg      
  inflating: data/normal/26.jpg      
  inflating: data/normal/27.jpg      
  inflating: data/normal/28.jpg      
  inflating: data/normal/29.jpg      
  inflating: data/normal/3.jpg       
  inflating: data/normal

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np
import os
from torch.utils.data import DataLoader, random_split
from PIL import Image


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#Бейзлайн - efficientnet_v2_s

In [18]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_dir, transform=None):
        self.dataset_dir = dataset_dir
        self.transform = transform
        self.images, self.labels = [], []
        for dirpath, _, filenames in os.walk(dataset_dir):
            for filename in filenames:
                label = 0 if 'normal' in dirpath else 1
                img_path = os.path.join(dirpath, filename)
                self.images.append(img_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [19]:
dataset = ImageDataset("/content/data", transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [20]:
def create_light_efficientnet():
    model = models.efficientnet_v2_s(weights="IMAGENET1K_V1")
    for layer in model.features:
        if isinstance(layer, nn.Conv2d):
            layer.out_channels = max(16, layer.out_channels // 2)

    num_ftrs = model.classifier[-1].in_features
    model.classifier[-1] = nn.Linear(num_ftrs, 2)

    return model

In [21]:
def train_model(model, num_epochs=10):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    best_acc = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss, correct = 0.0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()

        epoch_acc = correct / train_size
        print(f"Epoch {epoch+1}, Loss: {running_loss/train_size:.4f}, Accuracy: {epoch_acc:.4f}")
        scheduler.step()

        if epoch_acc > best_acc:
            best_acc = epoch_acc
            torch.save(model.state_dict(), "best_model.pth")

    return model, best_acc

In [22]:
def evaluate_model(model):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

    return correct / total

In [23]:
def model_size(model):
    torch.save(model.state_dict(), "temp.pth")
    size_mb = os.path.getsize("temp.pth") / (1024 * 1024)
    os.remove("temp.pth")
    return size_mb

In [24]:
light_efficientnet = create_light_efficientnet()
light_efficientnet, acc_effnet = train_model(light_efficientnet, num_epochs=10)
size_effnet = model_size(light_efficientnet)
test_acc_effnet = evaluate_model(light_efficientnet)

print(f"Light EfficientNetV2 - Точность: {test_acc_effnet:.4f}, Размер: {size_effnet:.2f} MB")

Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 92.3MB/s]


Epoch 1, Loss: 0.3944, Accuracy: 0.8245
Epoch 2, Loss: 0.2199, Accuracy: 0.9415
Epoch 3, Loss: 0.0739, Accuracy: 0.9681
Epoch 4, Loss: 0.0907, Accuracy: 0.9787
Epoch 5, Loss: 0.0873, Accuracy: 0.9840
Epoch 6, Loss: 0.0264, Accuracy: 1.0000
Epoch 7, Loss: 0.0070, Accuracy: 1.0000
Epoch 8, Loss: 0.0245, Accuracy: 0.9894
Epoch 9, Loss: 0.0698, Accuracy: 0.9734
Epoch 10, Loss: 0.0060, Accuracy: 1.0000
Light EfficientNetV2 - Точность: 0.9375, Размер: 77.79 MB


#Легкая модель

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [29]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [25]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(CustomCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(1)
        )

        self.classifier = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [30]:
def evaluate_model(model):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

    return correct / total

In [31]:
def train_model(model, num_epochs=20):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
    scheduler = lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=num_epochs)

    best_acc = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss, correct = 0.0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()
            running_loss += loss.item() * inputs.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()

        epoch_acc = correct / train_size
        print(f"Epoch {epoch+1}, Loss: {running_loss/train_size:.4f}, Accuracy: {epoch_acc:.4f}")

        if epoch_acc > best_acc:
            best_acc = epoch_acc
            torch.save(model.state_dict(), "best_model.pth")

    return model, best_acc

In [32]:
custom_cnn = CustomCNN()
custom_cnn, acc_custom = train_model(custom_cnn, num_epochs=20)
size_custom = model_size(custom_cnn)
test_acc_custom = evaluate_model(custom_cnn)

print(f"Custom CNN - Точность: {test_acc_custom:.4f}, Размер: {size_custom:.2f} MB")

Epoch 1, Loss: 0.4739, Accuracy: 0.7872
Epoch 2, Loss: 0.3259, Accuracy: 0.8564
Epoch 3, Loss: 0.3312, Accuracy: 0.8617
Epoch 4, Loss: 0.3413, Accuracy: 0.8670
Epoch 5, Loss: 0.3669, Accuracy: 0.8777
Epoch 6, Loss: 0.2622, Accuracy: 0.9043
Epoch 7, Loss: 0.3093, Accuracy: 0.8989
Epoch 8, Loss: 0.2630, Accuracy: 0.9043
Epoch 9, Loss: 0.4136, Accuracy: 0.8351
Epoch 10, Loss: 0.2867, Accuracy: 0.9096
Epoch 11, Loss: 0.3238, Accuracy: 0.8936
Epoch 12, Loss: 0.2672, Accuracy: 0.8936
Epoch 13, Loss: 0.2535, Accuracy: 0.9255
Epoch 14, Loss: 0.2631, Accuracy: 0.8883
Epoch 15, Loss: 0.3401, Accuracy: 0.8936
Epoch 16, Loss: 0.2625, Accuracy: 0.9096
Epoch 17, Loss: 0.2227, Accuracy: 0.9149
Epoch 18, Loss: 0.2205, Accuracy: 0.9202
Epoch 19, Loss: 0.2210, Accuracy: 0.9202
Epoch 20, Loss: 0.2293, Accuracy: 0.9043
Custom CNN - Точность: 0.7917, Размер: 3.88 MB
